# AUTOMATIC1111 Stable Diffusion WebUI 1.5 + ChilloutMix Checkpoint

More info: https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb

> Created by [@wibus-wee](https://github.com/wibus-wee)
>
> Reference: [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab)

In [ ]:
from IPython.display import display
import ipywidgets as widgets
import requests

#@title 选择模型 { display-mode: "form" }
endpoint = 'https://civitai.com/api/v1/models'
#@markdown 选择模型（ Others 需要后面填入模型ID 或 URL）
checkpointName = 'Chilloutmix' #@param ["Chilloutmix", "Sunshinemix", "grapefruit_hentai", "others"]
#@markdown 其他模型 （填入模型ID, 如 6424, 多个模型用英文逗号分隔）
checkpointID = '' #@param {type:"string"}
#@markdown 其他模型 （填入模型URL, 多个模型用英文逗号分隔）
checkpointURL = '' #@param {type:"string"}

#@markdown ---

#@markdown ## **LoRAs**

#@markdown Korean Doll Likeness `v1.0`
koreanDollLikenessv10 = True #@param {type:"boolean"}
#@markdown Korean Doll Likeness `v1.5`
koreanDollLikenessv15 = True #@param {type:"boolean"}
#@markdown St. Louis (Luxurious Wheels) (Azur Lane)
stLouis = True #@param {type:"boolean"}
#@markdown Girls' Frontline-OTs-14"lightning"
girlsFrontlineOTs14Lightning = True #@param {type:"boolean"}
#@markdown Yae Miko | Realistic Genshin (Mixed)
yaeMikoRealisticGenshinMixed = True #@param {type:"boolean"}
#@markdown Fashion Girl
fashionGirl = True #@param {type:"boolean"}
#@markdown Cute Girl mix4
cuteGirlMix4 = True #@param {type:"boolean"}
#@markdown More LoRA? (Enter the URL of the lora file, use comma to separate multiple URLs)
moreLoRA = "" #@param {type:"string"}

#@markdown ---

#@markdown ## **Textual Inversion**

#@markdown Ulzzang-6500 (Korean doll aesthetic)
ulzzang6500 = True #@param {type:"boolean"}
#@markdown Pure Eros Face
pureErosFace = True #@param {type:"boolean"}


defaultCheckpoint = {
    'Chilloutmix': '6424',
    'Sunshinemix': '9291',
    'grapefruit_hentai': '2583'
}

loraDownloadIDs = {
  'koreanDollLikenessv10': 'https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb/releases/download/koreanDollLikeness/koreanDollLikeness_v10.safetensors',
  'koreanDollLikenessv15': 'https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb/releases/download/koreanDollLikeness/koreanDollLikeness_v15.safetensors',
  'stLouis': 6669,
  'girlsFrontlineOTs14Lightning': 6525,
  'yaeMikoRealisticGenshinMixed': 8484,
  # 'chilloutMixss': 10850,
  'fashionGirl': {
    "id": 8217,
    "version": "v4.5"
  },
  'cuteGirlMix4': 14171,
}

textualInversionDownloadIDs = {
  'ulzzang6500': 8109,
  'pureErosFace': 4514,
}

downloadIds = []

if checkpointID != '':
    downloadIds = checkpointID.split(',')
if checkpointName != 'others':
        downloadIds.append(defaultCheckpoint[checkpointName])

globalDropdowns = []
globalVerions = []
globalNames = []
globalTexts = []
checkpoints = []
downloadLinks = []
customLoras = []
loraDownloadLinks = []

def text_on_submit(change):
    checkpoints[checkpoints.index(change['old'])] = change['new']

if checkpointURL != '':
    _downloadLinks = checkpointURL.split(',')
    for _downloadLink in _downloadLinks:
        checkpoints.append(_downloadLink.split('/')[-1])
        downloadLinks.append(_downloadLink)
        text = widgets.Text(value=_downloadLink.split('/')[-1], description=_downloadLink.split('/')[-1], disabled=False)
        text.observe(text_on_submit, names='value')
        form = widgets.VBox([text])
        display(form)
if moreLoRA != '':
    _downloadLinks = moreLoRA.split(',')
    for _downloadLink in _downloadLinks:
        customLoras.append(_downloadLink)
        loraDownloadLinks.append(_downloadLink)
        text = widgets.Text(value=_downloadLink.split('/')[-1], description=_downloadLink.split('/')[-1], disabled=False)
        text.observe(text_on_submit, names='value')
        form = widgets.VBox([text])
        display(form)


def showVerionOptions(downloadId):
    res = requests.get(endpoint + '/' + downloadId).json()
    globalNames.append(res['name'])
    versions = res['modelVersions']
    globalVerions.append(versions)
    options = []
    for version in versions:
        options.append(version['files'][0]['name'])
    dropdown = widgets.Dropdown(options=options, description=res['name'])
    globalDropdowns.append(dropdown)
    form = widgets.VBox([dropdown])
    display(form)

for downloadId in downloadIds:
    showVerionOptions(downloadId)

def on_button_clicked(b):
    downloadLink = None
    for dropdown in globalDropdowns:
        checkpoint = dropdown.value
        versions = globalVerions[globalDropdowns.index(dropdown)]
        for version in versions:
            if version['files'][0]['name'] == checkpoint:
                downloadLink = version['files'][0]['downloadUrl']
                break
        if downloadLink is None:
            print('Error: downloadLink not assigned')
            return
        checkpoints.append(checkpoint)
        downloadLinks.append(downloadLink)

    print("已选择模型: " + str(checkpoints) + " <===> " + str(downloadLinks))
    print("已选择LoRA: " + str(customLoras) + " <===> " + str(loraDownloadLinks))
    %store checkpoints
    %store downloadLinks


button = widgets.Button(description='Use it!')
button.on_click(on_button_clicked)
display(button)

In [ ]:
#@title 2. 检查 GPU & 检查环境

#@markdown 此步骤将检查你的 GPU 是否支持 xformers，同时会检查是否为 Paperspace 平台（仅针对 M4000 GPU 做了判断，因此付费 GPU 可能在此处将会出现逻辑判断错误的情况，你可能需要自行勾选为 Paperspace 平台）。

import os, subprocess
paperspace_m4000 = False
#@markdown 是否为 Paperspace 平台
isPaperspace = False #@param {type:"boolean"}
try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    if 'M4000' in subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8'):
        print("WARNING: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
        paperspace_m4000 = True
        isPaperspace = True
    else:
        print("你正在使用的是合适的 GPU - " + subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8') + "。")
        print("平台: Paperspace" if isPaperspace else "使用平台: Colab")
except:
    print("似乎没有 GPU 可用。请检查你的运行时类型。")
    exit()

rootDir = isPaperspace and '/tmp' or '/content'
stableDiffusionWebUIInstalled = os.path.exists(rootDir + '/stable-diffusion-webui')
%store rootDir
%store paperspace_m4000 
%store isPaperspace
%store stableDiffusionWebUIInstalled

In [ ]:
import requests
#@title 3. 第一次使用 - 安装依赖并启动

#@markdown ## **Extensions**
#@markdown > 你可以在安装时选择安装扩展，扩展将会在启动时自动安装。如果你想要在启动后再安装扩展，你可能需要重启运行时。

#@markdown 安装 xformer 包 (仅在 Paperspace M4000 GPU 平台上运行时需要选择，若选择则会构建 xformers 包).
xformersInstall = True #@param {type:"boolean"}
#@markdown 安装 ControlNet 扩展
controlNetExtension = True #@param {type:"boolean"}
#@markdown 安装 OpenPose Editor 扩展
openPoseExtension = True #@param {type:"boolean"}
#@markdown 安装 Civitai Browser 扩展
civitaiBrowserExtension = True #@param {type:"boolean"}
#@markdown 安装 HuggingFace 扩展
huggingFaceExtension = True #@param {type:"boolean"}
#@markdown 安装 Images Browser 扩展
imagesBrowserExtension = True #@param {type:"boolean"}
#@markdown 安装 Additional Networks 扩展
additionalNetworksExtension = True #@param {type:"boolean"}
#@markdown 安装 Deforum 扩展
deforumExtension = False #@param {type:"boolean"}
#@markdown 安装 Kohya sd-scripts 扩展 ( 启动后再安装需要重启 )
kohyaExtension = False #@param {type:"boolean"}
#@markdown 安装 DreamBooth 扩展
dreamBoothExtension = False #@param {type:"boolean"}

#@markdown ---

#@markdown ## **Others**

#@markdown Styles CSV File URL
stylesCSVFileURL = "" #@param {type:"string"}
#@markdown Use [Cloudflared](https://github.com/cloudflare/cloudflared) instead of Gradio share
useCloudflared = False #@param {type:"boolean"}

#@markdown ---

#@markdown ## **Startup Options**

#@markdown API Support
apiSupport = True #@param {type:"boolean"}
#@markdown Cors Allow Origins
corsAllowOrigins = "*" #@param {type:"string"}

def getLatestModelDownloadURL(id):
  try:
    if type(id) == int:
      res = requests.get(endpoint + '/' + str(id)).json()
      latest = res['modelVersions'][0]
      downloadLink = latest['files'][0]['downloadUrl']
      name = latest['files'][0]['name']
      return {
        'url': downloadLink,
        'name': name
      }
    else:
      return {
        'url': id,
        'name': id.split('/')[-1]
      }
  except:
    print("Lora model " + str(id) + " not found. Skip.")
    return None

def getSpecificModelDownloadURL(id, version):
  try:
    if type(id) == int:
      res = requests.get(endpoint + '/' + str(id)).json()
      for modelVersion in res['modelVersions']:
        if modelVersion['name'] == version:
          # if modelVersion["baseModel"] != "SD 1.5":
          #   print("Lora model " + str(id) + " is not SD 1.5, may not work. Skip.")
          #   return None
          downloadLink = modelVersion['files'][0]['downloadUrl']
          name = modelVersion['files'][0]['name']
          return {
            'url': downloadLink,
            'name': name
          }
    else:
      return {
        'url': id,
        'name': id.split('/')[-1]
      }
  except:
    print("Lora model " + str(id) + " version " + version + " not found. Skip.")
    return None

def getLoraDownloadURLs():
  downloadURLs = []
  for key in loraDownloadIDs:
    if not eval(key): # skip if not selected
      continue
    if type(loraDownloadIDs[key]) is int:
      downloadURLs.append(getLatestModelDownloadURL(loraDownloadIDs[key]))
    elif type(loraDownloadIDs[key]) is dict: # {'id': 123, 'version': 'v1.0'}
      downloadURLs.append(getSpecificModelDownloadURL(loraDownloadIDs[key]['id'], loraDownloadIDs[key]['version']))
    elif type(loraDownloadIDs[key]) is str: # url
      downloadURLs.append({ 'url': loraDownloadIDs[key], 'name': loraDownloadIDs[key].split('/')[-1] })
  downloadURLs = [x for x in downloadURLs if x is not None] # remove None
  for custom in customLoras:
    downloadURLs.append({ 'url': loraDownloadLinks[customLoras.index(custom)], 'name': custom })
  return downloadURLs

def getTextualInversionDownloadURLs():
  downloadURLs = []
  for key in textualInversionDownloadIDs:
    if not eval(key): # skip if not selected
      continue
    if type(textualInversionDownloadIDs[key]) is int:
      downloadURLs.append(getLatestModelDownloadURL(textualInversionDownloadIDs[key]))
    elif type(textualInversionDownloadIDs[key]) is dict: # {'id': 123, 'version': 'v1.0'}
      downloadURLs.append(getSpecificModelDownloadURL(textualInversionDownloadIDs[key]['id'], textualInversionDownloadIDs[key]['version']))
    elif type(textualInversionDownloadIDs[key]) is str: # url
      downloadURLs.append({ 'url': textualInversionDownloadIDs[key], 'name': textualInversionDownloadIDs[key].split('/')[-1] })
  downloadURLs = [x for x in downloadURLs if x is not None]
  return downloadURLs
    

loraDownloadURLs = getLoraDownloadURLs()
textualInversionDownloadURLs = getTextualInversionDownloadURLs()


%store -r paperspace_m4000 
%store -r isPaperspace
%store -r rootDir 
%store -r checkpoints
%store -r downloadLinks
%store -r stableDiffusionWebUIInstalled

import subprocess

!apt-get -y install -qq aria2
ariaInstalled = False

try:
    subprocess.run(['aria2c', '--version'], stdout=subprocess.PIPE)
    ariaInstalled = True
except:
    pass

if paperspace_m4000:
  if xformersInstall:
    !pip install ninja
    !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers
    !pip install -q --pre triton
else:
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  !pip install -q --pre triton
  

!git clone -b v2.0 https://github.com/camenduru/stable-diffusion-webui {rootDir}/stable-diffusion-webui
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O {rootDir}/stable-diffusion-webui/scripts/run_n_times.py
if deforumExtension:
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {rootDir}/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
if imagesBrowserExtension:
  !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
if huggingFaceExtension:
  !git clone https://github.com/camenduru/stable-diffusion-webui-huggingface {rootDir}/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
if civitaiBrowserExtension:
  !git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser {rootDir}/stable-diffusion-webui/extensions/sd-civitai-browser
if openPoseExtension:
  !git clone https://github.com/camenduru/openpose-editor {rootDir}/stable-diffusion-webui/extensions/openpose-editor
if controlNetExtension:
  !git clone https://github.com/Mikubill/sd-webui-controlnet {rootDir}/stable-diffusion-webui/extensions/sd-webui-controlnet
if additionalNetworksExtension:
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks
if kohyaExtension:
  !git clone https://github.com/ddpn08/kohya-sd-scripts-webui.git {rootDir}/stable-diffusion-webui/extensions/kohya-sd-scripts-webui
if dreamBoothExtension:
  !git clone https://github.com/d8ahazard/sd_dreambooth_extension {rootDir}/stable-diffusion-webui/extensions/sd_dreambooth_extension

if isPaperspace:
  %cd /stable-diffusion-webui
else:
  %cd {rootDir}/stable-diffusion-webui


webuiControlNetModels = [
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors",
  "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors",
]
annotatorLink = [
  "https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth",
  "https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth",
]

def ariaDownload(downloadLink, checkpoint, path):
  if (type(downloadLink) == list and type(checkpoint) == list):
    for i in downloadLink:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {i} -d {path} -o {checkpoint[downloadLink.index(i)]}
  else:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {downloadLink} -d {path} -o {checkpoint}
def wgetDownload(downloadLink, checkpoint, path):
  if (type(downloadLink) == list and type(checkpoint) == list):
    for i in downloadLink:
      !wget -c {i} -P {path} -O {checkpoint[downloadLink.index(i)]}
  else:
    !wget -c {downloadLink} -P {path} -O {checkpoint}
def autoDetectDownload(downloadLink, checkpoint, path):
  if ariaInstalled:
    ariaDownload(downloadLink, checkpoint, path)
  else:
    wgetDownload(downloadLink, checkpoint, path)

autoDetectDownload(downloadLinks, checkpoints, rootDir + "/stable-diffusion-webui/models/Stable-diffusion")
if controlNetExtension:
  for model in webuiControlNetModels:
    autoDetectDownload(model, model.split('/')[-1], rootDir + "/stable-diffusion-webui/extensions/sd-webui-controlnet/models")
  for model in annotatorLink:
    autoDetectDownload(model, model.split('/')[-1], rootDir + "/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator")
for model in loraDownloadURLs:
  autoDetectDownload(model["url"], model["name"], rootDir + "/stable-diffusion-webui/models/Lora")
for model in textualInversionDownloadURLs:
  autoDetectDownload(model["url"], model["name"], rootDir + "/stable-diffusion-webui/embeddings")
if stylesCSVFileURL:
  autoDetectDownload(stylesCSVFileURL, "styles.csv", rootDir + "/stable-diffusion-webui")

if additionalNetworksExtension:
  !ln -s {rootDir}/stable-diffusion-webui/models/Lora {rootDir}/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora


stableDiffusionWebUIInstalled = True
%store stableDiffusionWebUIInstalled

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py
if dreamBoothExtension:
  !export REQS_FILE="./extensions/sd_dreambooth_extension/requirements.txt"

backgroundProcess = ''

if useCloudflared:
   backgroundProcess = 'cloudflared tunnel --url 127.0.0.1:7860'
  # !cloudflared tunnel --url 127.0.0.1:7860
if backgroundProcess:
   backgroundProcess = backgroundProcess + ' &'

if useCloudflared:
  !echo "Installing cloudflared"
  !curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

!{backgroundProcess} python launch.py --enable-insecure-extension-access {'' if paperspace_m4000 and not xformersInstall else '--xformers'} {'--api --cors-allow-origins "*" --listen' if corsAllowOrigins else ''} {'--share' if not backgroundProcess else ''}

In [ ]:

#@title 4. 重启 - 重新启动 Stable Diffusion WebUI

%store -r paperspace_m4000 
%store -r rootDir 
%store -r stableDiffusionWebUIInstalled

if not stableDiffusionWebUIInstalled:
    print("请先运行安装 Stable Diffusion WebUI 的单元格。")
    exit()

%cd {rootDir}/stable-diffusion-webui
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' {rootDir}/stable-diffusion-webui/launch.py
!sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {rootDir}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {rootDir}/stable-diffusion-webui/launch.py

!{backgroundProcess} python launch.py --enable-insecure-extension-access {'' if paperspace_m4000 and not xformersInstall else '--xformers'} {'--api --cors-allow-origins "*" --listen' if corsAllowOrigins else ''} {'--share' if not backgroundProcess else ''}

In [ ]:
#@title 5. 导出 - 导出生成的图片
%store -r rootDir 

from pathlib import Path
import os, subprocess

export_storage_dir = Path(rootDir, 'export')
export_storage_dir.mkdir(exist_ok=True)

!if [ $(dpkg-query -W -f='${Status}' p7zip-full 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y p7zip-full; fi # install 7z if it isn't already installed
from datetime import datetime
datetime_str = datetime.now().strftime('%m-%d-%Y_%H-%M-%S')
%cd "{export_storage_dir}"
!mkdir -p "{datetime_str}/log"
!cd "{rootDir}/stable-diffusion-webui/log" && mv * "{export_storage_dir / datetime_str / 'log'}"
!cd "{rootDir}/stable-diffusion-webui/outputs" && mv * "{export_storage_dir / datetime_str}"
s = subprocess.run(f'find "{Path(export_storage_dir, datetime_str)}" -type d -name .ipynb_checkpoints -exec rm -rv {{}} +', shell=True)
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "{datetime_str}.7z" "{export_storage_dir / datetime_str}"